In [1186]:
import tensorflow as tf
import numpy as np
import random
import glob
import pandas as pd
import pickle

In [1187]:
#Params
kld_reg= 1
adl_reg=1

fdim=16
zdim=16
sigma=1.3
past_length=8
future_length=12
data_scale=1.86

enc_past_size=(past_length*2,512,256,fdim)
enc_dest_size=(2,8,16,fdim)

enc_latent_size=(2*fdim,8,50,2*zdim)
dec_size=(fdim + zdim,1024,512,1024,2)

predictor_size=(2*fdim + 2,1024,512,256,2*(future_length-1))

non_local_theta_size = (2*fdim + 2,256,128,64,128)
non_local_phi_size=(2*fdim + 2,256,128,64,128)
non_local_g_size=(2*fdim + 2,256,128,64,2*fdim + 2)
nonlocal_pools=3
learning_rate=0.0003

In [1188]:
def loadData(file_path: str):
  npz = np.load(file_path, allow_pickle=True)
  return npz['observations'], npz['obs_speed'], npz['targets'], npz[
      'target_speed'], npz['mean'], npz['std']

In [1189]:
class Dense(tf.Module):
  def __init__(self, input_dim, output_size, name=None):
    super(Dense, self).__init__(name=name)
    self.w = tf.Variable(tf.random.uniform([input_dim, output_size],-(1.0/input_dim)**0.5,(1.0/input_dim)**0.5 ),name='w',dtype=tf.float32,trainable=True)
    self.b = tf.Variable(tf.random.uniform([output_size],-(1.0/input_dim)**0.5,(1.0/input_dim)**0.5 ), name='b',dtype=tf.float32,trainable=True)
  def __call__(self, x):
    #x = tf.constant(x,dtype=tf.float32)
    y = tf.matmul(x, self.w) + self.b
    return y

class FullyConnectedNeuralNet(tf.Module):
  def __init__(self,sizes, name=None):
    super(FullyConnectedNeuralNet, self).__init__(name=name)
    self.layers = []
    with self.name_scope:
      for i in range(len(sizes)-1):
        self.layers.append(Dense(input_dim=sizes[i], output_size=sizes[i+1]))
  @tf.Module.with_name_scope
  def __call__(self, x):
    for i,layer in enumerate(self.layers):
        x=layer(x)
        if i != len(self.layers)-1:
            x = tf.nn.relu(x)
    return x

In [1190]:
class MainModel(tf.Module):
    def __init__(self,name=None):
        super(MainModel, self).__init__(name=name)

        self.zdim = zdim
        self.sigma = sigma
        self.nonlocal_pools = nonlocal_pools
        


        self.pastEncoder = FullyConnectedNeuralNet(enc_past_size)

        self.destEncoder = FullyConnectedNeuralNet(enc_dest_size)

        self.latentDistributionEncoder = FullyConnectedNeuralNet(enc_latent_size)

        self.latentDistributionDecoder = FullyConnectedNeuralNet(dec_size)


        self.nonLocalTheta = FullyConnectedNeuralNet(non_local_theta_size)
        self.nonLocalPhi = FullyConnectedNeuralNet(non_local_phi_size)
        self.nonLocalG = FullyConnectedNeuralNet(non_local_g_size)
        
        self.predictorNetwork = FullyConnectedNeuralNet(predictor_size)

    def forward(self, x, initial_pos, dest=[], mask =[] ):

        if len(dest):
            self.training=True
        else:
            self.training=False
        
        # encode
        traj_past_ftr = self.pastEncoder(x)
        #print(f"ftraj max {ftraj.numpy().max()}")
        if not self.training:
            z = tf.random.normal((x.shape[0], self.zdim),0,self.sigma)

        else:
            dest_ftr = self.destEncoder(dest)
            #print(f"dest_features Max {dest_features.numpy().max()}")

            concat_ftr =tf.concat((traj_past_ftr, dest_ftr), axis = 1)
            latent =  self.latentDistributionEncoder(concat_ftr)
            mu = latent[:, 0:self.zdim] # 2-d array
            logvar =  latent[:, self.zdim:] # 2-d array

            var = tf.math.exp(logvar*0.5)
            #print(f"var {var}")
            eps = tf.random.normal(var.shape)
            z = eps*var + mu
            #print(f"z -> {z}")


        latentDistributionDecoder_input = tf.concat((traj_past_ftr, z), axis = 1)
        generated_dest = self.latentDistributionDecoder(latentDistributionDecoder_input)
        #generated_dest = tf.where(tf.math.is_nan(generated_dest), tf.zeros_like(generated_dest), generated_dest)

        if self.training:
            generated_dest_ftr = self.destEncoder(generated_dest)
            #generated_dest_ftr = tf.where(tf.math.is_nan(generated_dest_ftr), tf.zeros_like(generated_dest_ftr), generated_dest_ftr)
            #print(f"{ tf.math.reduce_any(tf.math.is_nan(generated_dest_ftr))}")

            prediction_ftr = tf.concat((traj_past_ftr, generated_dest_ftr,initial_pos), axis = 1)
            for i in range(self.nonlocal_pools):
                prediction_ftr = self.nonLocalSocialPooling(prediction_ftr, mask)
            pred_future = self.predictorNetwork(prediction_ftr)
        
            return generated_dest, mu, logvar, pred_future
        else:
            return generated_dest
    
    def nonLocalSocialPooling(self, feat, mask):
        # N,C
        theta_x = self.nonLocalTheta(feat)
        # C,N
        phi_x = tf.transpose(self.nonLocalPhi(feat))

        # f_ij = (theta_i)^T(phi_j), (N,N)
        f = tf.matmul(theta_x , phi_x)

        # f_weights_i =  exp(f_ij)/(\sum_{j=1}^N exp(f_ij))
        f_weights =tf.nn.softmax(f, axis = -1)
        # setting weights of non neighbours to zero
        f_weights = f_weights * mask
        pooled_g = self.nonLocalG(feat)
        
        #print(f"f_weights {f_weights.shape}")
        #print(f"self.pooled_g(feat) {pooled_g.shape}")
        
        # rescaling row weights to 1
        f_weights = tf.math.l2_normalize(f_weights,axis=1)
        #print(f"f_weights {f_weights.shape}")

        # ith row of all_pooled_f = \sum_{j=1}^N f_weights_i_j * g_row_j
        pooled_f = tf.matmul(f_weights, pooled_g)

        return pooled_f + feat

    def predict(self, past, generated_dest, mask, initial_pos):
        
        traj_past_ftr = self.pastEncoder(past)
        generated_dest_ftr = self.destEncoder(generated_dest)
        prediction_ftr = tf.concat((traj_past_ftr, generated_dest_ftr,initial_pos), axis = 1)
        for i in range(self.nonlocal_pools):
            prediction_ftr = self.nonLocalSocialPooling(prediction_ftr, mask)   
        future_traj = self.predictorNetwork(prediction_ftr)
        return future_traj

In [1191]:
def calculate_loss(dest, dest_rec, mean, log_var, future, future_rec):
    
    rcl = tf.math.reduce_mean(tf.keras.metrics.mean_squared_error(dest, dest_rec))
    adl = tf.math.reduce_mean(tf.keras.metrics.mean_squared_error(future, future_rec))

    kld = -0.5 * tf.math.reduce_sum(1 + log_var - mean**2 - tf.math.exp(log_var))

    return rcl, kld, adl

In [1192]:
def next_batch(X,batchSize):
    start = random.randint(0, len(X)-batchSize)
    return X[start:start+batchSize]

In [1193]:
def train(model,optimizer):
    trajectory_batches,mask_batches,initial_pos_batches = loadDataSocial('./social_pool_data/train_all_512_0_100.pickle',set_name="train")
    
    train_loss = 0
    total_rcl, total_kld, total_adl = 0, 0, 0
    
    for i, (traj, mask, initial_pos) in enumerate(zip(trajectory_batches,mask_batches,initial_pos_batches)):
        traj -= traj[:, :1, :]
        traj *= data_scale
        x = traj[:, :past_length, :]
        y = traj[:, past_length:, :]

        x = x.reshape(-1, x.shape[1]*x.shape[2]) # (x,y,x,y ... )
        dest = y[:, -1, :]
        future = y[:, :-1, :].reshape(y.shape[0],-1)
        #x.astype(np.float64)
        #print(f"trajx-> {trajx.shape}")
        
        #print(f"X shape -> {x.shape}")
        #print(f"initial_pos -> {initial_pos.shape}")
        #print(f"dest shape -> {dest.shape}")
        #print(f"mask shape -> {mask.shape}")
         
        with tf.GradientTape() as tape:
            x=tf.constant(x,dtype=tf.float32)
            initial_pos=tf.constant(initial_pos,dtype=tf.float32)
            dest=tf.constant(dest,dtype=tf.float32)

            tape.watch(x)
            tape.watch(initial_pos)
            tape.watch(dest)

            dest_rec, mu, var, future_rec = model.forward(x, initial_pos, dest=dest, mask=mask)
            #print(f"dest_recon {dest_recon}")
            #print(f"mu {mu}")
            #print(f"var {var}")
            #print(f"interpolated_future {interpolated_future}") 
            rcl, kld, adl = calculate_loss(dest, dest_rec, mu, var, future, future_rec)   
            loss = rcl + kld * kld_reg + adl * adl_reg
            #print(f"loss -> {loss}")
        grad_sub = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grad_sub, model.trainable_variables))
       
                    
        #print(f"total Loss {loss}")
        #print(f"rcl Loss {rcl}")
        #print(f"kld Loss {kld}")
        #print(f"adl Loss {adl}")
        train_loss+=loss
        total_rcl+=rcl
        total_kld+=kld
        total_adl+=adl
    return train_loss, total_rcl, total_kld, total_adl
                

In [1194]:
def test(model, best_of_n = 1):
    trajectory_batches,mask_batches,initial_pos_batches = loadDataSocial('./social_pool_data/test_all_4096_0_100.pickle',set_name="test")
    for i, (traj, mask, initial_pos) in enumerate(zip(trajectory_batches,mask_batches,initial_pos_batches)):
   
        traj -= traj[:, :1, :]
        traj *= data_scale
        x = traj[:, :past_length,:]
        y = traj[:, past_length:,:]

        x = x.reshape(-1, x.shape[1]*x.shape[2])

        dest = y[:, -1, :]
    
        destination_errors = []
        dectination_recs = []
    
        for _ in range(best_of_n):
            x=tf.constant(x,dtype=tf.float32)
            initial_pos=tf.constant(initial_pos,dtype=tf.float32)
            dest_rec = model.forward(x, initial_pos)
            dectination_recs.append(np.array(dest_rec))

            error = np.linalg.norm(dest_rec - dest, axis = 1)
            destination_errors.append(error)

        destination_errors = np.array(destination_errors)
        dectination_recs = np.array(dectination_recs)
        # average error
        avg_dest_error = np.mean(destination_errors)

        indices = np.argmin(destination_errors, axis = 0)

        best_dest = dectination_recs[indices,np.arange(x.shape[0]),  :]

        # taking the minimum error out of all guess
        dest_error = np.mean(np.min(destination_errors, axis = 0))

        future_dest = model.predict(x, best_dest, mask, initial_pos)
        # final overall prediction
        predicted_future = np.concatenate((future_dest, best_dest), axis = 1)
        predicted_future = np.reshape(predicted_future, (-1, future_length, 2))
        # ADE error
        overall_error = np.mean(np.linalg.norm(y - predicted_future, axis = 2))

        overall_error /= data_scale
        dest_error /= data_scale
        avg_dest_error /= data_scale
        #print('Test time error in destination best: {:0.3f} and mean: {:0.3f}'.format(dest_error, avg_dest_error))
        #print('Test time error overall (ADE) best: {:0.3f}'.format(overall_error))

    return overall_error, dest_error, avg_dest_error

            

In [1195]:
def run_train():
    epochs = 1000
    batchSize=100
    model=MainModel()
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    N=20
    best_test_loss = 50 # start saving after this threshold
    best_endpoint_loss = 50
    for epo in range(epochs):
        train_loss, rcl, kld, adl = train(model,optimizer)
        test_loss, final_point_loss_best, final_point_loss_avg = test(model, best_of_n = N)
                
        if best_test_loss > test_loss:
            print("Epoch: ", epo+1)
            print('################## BEST PERFORMANCE {:0.2f} ########'.format(test_loss))
            best_test_loss = test_loss
        

        if final_point_loss_best < best_endpoint_loss:
            best_endpoint_loss = final_point_loss_best

        print("Train Loss", train_loss)
        print("RCL", rcl)
        print("KLD", kld)
        print("ADL", adl)
        print("Test ADE", test_loss)
        print("Test Average FDE (Across  all samples)", final_point_loss_avg)
        print("Test Min FDE", final_point_loss_best)
        print("Test Best ADE Loss So Far (N = {})".format(N), best_test_loss)
        print("Test Best Min FDE (N = {})".format(N), best_endpoint_loss)

In [1196]:
def loadDataSocial(load_name,set_name="train", id=False):
    
    with open(load_name, 'rb') as f:
        data = pickle.load(f)
    traj, masks = data
    traj_new = []

    if id==False:
        for t in traj:
            t = np.array(t)
            t = t[:,:,2:]
            traj_new.append(t)
            if set_name=="train":
            #augment training set with reversed tracklets...
                reverse_t = np.flip(t, axis=1).copy()
                traj_new.append(reverse_t)
    else:
        for t in traj:
            t = np.array(t)
            traj_new.append(t)
            if set_name=="train":
                #augment training set with reversed tracklets...
                reverse_t = np.flip(t, axis=1).copy()
                traj_new.append(reverse_t)
    masks_new = []
    
    for m in masks:
        masks_new.append(m)
        if set_name=="train":
            #add second time for the reversed tracklets...
            masks_new.append(m)

    traj_new = np.array(traj_new)
    masks_new = np.array(masks_new)
    trajectory_batches = traj_new.copy()
    mask_batches = masks_new.copy()
    
    initial_pos_batches = np.array(initial_pos(trajectory_batches)) # for relative positioning
    return trajectory_batches,mask_batches,initial_pos_batches

def initial_pos(traj_batches):
    batches = []
    for b in traj_batches:
        starting_pos = b[:,7,:].copy()/1000 #starting pos is end of past, start of future. scaled down.
        batches.append(starting_pos)
    return batches

In [ ]:
run_train()

<ipython-input-1196-47f677a6bda1>:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  traj_new = np.array(traj_new)
<ipython-input-1196-47f677a6bda1>:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  masks_new = np.array(masks_new)
<ipython-input-1196-47f677a6bda1>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  initial_pos_batches = n

Train Loss tf.Tensor(3376031.2, shape=(), dtype=float32)
RCL tf.Tensor(2161432.5, shape=(), dtype=float32)
KLD tf.Tensor(34106.88, shape=(), dtype=float32)
ADL tf.Tensor(1180492.0, shape=(), dtype=float32)
Test ADE 116.40176981676316
Test Average FDE (Across  all samples) 163.70176910072243
Test Min FDE 163.4874241326445
Test Best ADE Loss So Far (N = 20) 50
Test Best Min FDE (N = 20) 50
Train Loss tf.Tensor(2817523.0, shape=(), dtype=float32)
RCL tf.Tensor(1942469.0, shape=(), dtype=float32)
KLD tf.Tensor(5911.9414, shape=(), dtype=float32)
ADL tf.Tensor(869142.56, shape=(), dtype=float32)
Test ADE 104.80539569364748
Test Average FDE (Across  all samples) 162.16952416204637
Test Min FDE 161.95255402595765
Test Best ADE Loss So Far (N = 20) 50
Test Best Min FDE (N = 20) 50
Train Loss tf.Tensor(2777374.5, shape=(), dtype=float32)
RCL tf.Tensor(1932778.1, shape=(), dtype=float32)
KLD tf.Tensor(2462.7954, shape=(), dtype=float32)
ADL tf.Tensor(842133.9, shape=(), dtype=float32)
Test ADE 1

Train Loss tf.Tensor(2718949.5, shape=(), dtype=float32)
RCL tf.Tensor(1898998.0, shape=(), dtype=float32)
KLD tf.Tensor(80.58594, shape=(), dtype=float32)
ADL tf.Tensor(819870.8, shape=(), dtype=float32)
Test ADE 103.80255637462216
Test Average FDE (Across  all samples) 160.3341051327285
Test Min FDE 159.8699918357275
Test Best ADE Loss So Far (N = 20) 50
Test Best Min FDE (N = 20) 50
Train Loss tf.Tensor(2717305.5, shape=(), dtype=float32)
RCL tf.Tensor(1898154.1, shape=(), dtype=float32)
KLD tf.Tensor(82.37522, shape=(), dtype=float32)
ADL tf.Tensor(819068.8, shape=(), dtype=float32)
Test ADE 103.90530240800561
Test Average FDE (Across  all samples) 160.5057911206317
Test Min FDE 160.01518003402217
Test Best ADE Loss So Far (N = 20) 50
Test Best Min FDE (N = 20) 50
Train Loss tf.Tensor(2717067.5, shape=(), dtype=float32)
RCL tf.Tensor(1898070.4, shape=(), dtype=float32)
KLD tf.Tensor(86.39893, shape=(), dtype=float32)
ADL tf.Tensor(818910.44, shape=(), dtype=float32)
Test ADE 103.81

Train Loss tf.Tensor(2702154.0, shape=(), dtype=float32)
RCL tf.Tensor(1888911.6, shape=(), dtype=float32)
KLD tf.Tensor(314.16614, shape=(), dtype=float32)
ADL tf.Tensor(812928.6, shape=(), dtype=float32)
Test ADE 103.72066217253715
Test Average FDE (Across  all samples) 160.5959984564012
Test Min FDE 159.63646673387095
Test Best ADE Loss So Far (N = 20) 50
Test Best Min FDE (N = 20) 50
Train Loss tf.Tensor(2700871.2, shape=(), dtype=float32)
RCL tf.Tensor(1888022.9, shape=(), dtype=float32)
KLD tf.Tensor(332.88275, shape=(), dtype=float32)
ADL tf.Tensor(812515.2, shape=(), dtype=float32)
Test ADE 103.63827606875196
Test Average FDE (Across  all samples) 160.5100898332493
Test Min FDE 159.52156641150032
Test Best ADE Loss So Far (N = 20) 50
Test Best Min FDE (N = 20) 50
Train Loss tf.Tensor(2700824.0, shape=(), dtype=float32)
RCL tf.Tensor(1887924.6, shape=(), dtype=float32)
KLD tf.Tensor(360.8471, shape=(), dtype=float32)
ADL tf.Tensor(812538.7, shape=(), dtype=float32)
Test ADE 103.

Train Loss tf.Tensor(2686563.0, shape=(), dtype=float32)
RCL tf.Tensor(1873063.6, shape=(), dtype=float32)
KLD tf.Tensor(3752.9827, shape=(), dtype=float32)
ADL tf.Tensor(809746.44, shape=(), dtype=float32)
Test ADE 103.83143191307587
Test Average FDE (Across  all samples) 160.302488265499
Test Min FDE 157.45987430695564
Test Best ADE Loss So Far (N = 20) 50
Test Best Min FDE (N = 20) 50
Train Loss tf.Tensor(2684175.8, shape=(), dtype=float32)
RCL tf.Tensor(1869566.2, shape=(), dtype=float32)
KLD tf.Tensor(4909.2207, shape=(), dtype=float32)
ADL tf.Tensor(809700.56, shape=(), dtype=float32)
Test ADE 103.50442190021366
Test Average FDE (Across  all samples) 160.19205072874664
Test Min FDE 157.08250640540993
Test Best ADE Loss So Far (N = 20) 50
Test Best Min FDE (N = 20) 50
Train Loss tf.Tensor(2679565.8, shape=(), dtype=float32)
RCL tf.Tensor(1864760.6, shape=(), dtype=float32)
KLD tf.Tensor(6611.816, shape=(), dtype=float32)
ADL tf.Tensor(808193.2, shape=(), dtype=float32)
Test ADE 10